# Analyzing mother-infant metagenomes to find evidence for plasmid transfer through single-nucleotide variants (SNVs)

The purpose of the following workflow is to use read recruitment results obtained from multiple mother-infant gut metagenomes using pBI143 Version 1 sequence to investigate whether unqiue SNVs occur primarily between family members.

If you are planning to reproduce this workflow, I would suggest you to download [this jupyter notebook](https://merenlab.org/data/pBI143/files/pBI143_SNVs.ipynb) file on your computer, and follow it from within your local jupyter environment. If you wish to do that, all you need to do is the following, assuming you are in an `anvio-dev` installed environment (the installation instructions for anvio-dev is [here](https://anvio.org/install/)):

* Download [this file](https://merenlab.org/data/pBI143/files/pBI143_SNVs.ipynb) on your computer.
* In your terminal go to the directory in which you have downloaded the file.
* In your terminal type `jupyter notebook`, and select `pBI143_SNVs.ipynb` to start.

Alternatively, you can read through the following steps and look at the output files the workflow reports.

## Acquiring the data pack

The primary input for this analysis is anvi'o project files for four mother-infant datasets from Finalnd, Italy, Sweeden, and the United Stats. The generation of these project files is very straightforward with the program anvi-run-workflow, to which we essentially provided the plasmid sequence and a list of metagenomes of interest. The program anvi-run-workflow simply (1) recruited reads using the plasmid sequence from each metagenome it was given, (2) profiled each read recruitment result using the program anvi-profile, and finally (3) merged all single profile databases using the program anvi-merge. We then moved these final anvi'o project files into four directories (which we will download for reproducibility in a second), where each directory contained a single contigs-db and a single merged profile-db that will enable us to perform the analyses down below. 

Let's start with downloading the data pack, which is at [doi:10.6084/m9.figshare.22298215](https://doi.org/10.6084/m9.figshare.22298215). In this jupyter notebook environment, I will use an anvi'o function to directly download it to my work directory:

In [ ]:
# from anvi'o utils library import necessary functions
from anvio.utils import download_file, gzip_decompress_file, tar_extract_file

# instruct jupyter notebook to download the datapack
download_file('https://figshare.com/ndownloader/files/39659905',
              output_file_path='MOTHER_INFANT_pBI143_POP_GEN.tar.gz')

# if we are here, the download is finished. now we will
# first decompress it (and get rid of the original file
# while at it).
gzip_decompress_file('MOTHER_INFANT_pBI143_POP_GEN.tar.gz',
                     keep_original=False)

# and finally untar it so we have a clean directory:
tar_extract_file('MOTHER_INFANT_pBI143_POP_GEN.tar',
                 output_file_path='.',
                 keep_original=False)

Running the lines above must have created a new directory called `MOTHER_INFANT_pBI143_POP_GEN` in our work directory. Run the `ls` command to confirm:

In [ ]:
ls

Where the datapack directory should contain four folders as promised:

In [ ]:
ls MOTHER_INFANT_pBI143_POP_GEN

With anvi'o contigs-db and profile-db files in them:

In [ ]:
ls MOTHER_INFANT_pBI143_POP_GEN/*/

Good. Now we have the raw recruitment results described as anvi'o project files. Now we will change our work directory to the root of the data pack, and start playing with these data:

In [ ]:
import os
os.chdir('MOTHER_INFANT_pBI143_POP_GEN')

## Analysis

Unlike the vast majority of data analyses we do with anvi'o using [anvi'o programs](https://anvio.org/help/), here we will perform this analysis by accessing anvi'o libraries directly from within Python scripts we will implement below. Many of these steps use anvi'o functionality that is accessible through two anvi'o programs:

* [anvi-gen-variability-profile](https://anvio.org/m/anvi-gen-variability-profile) (which is a program that gives us access to nucleotide, codon, or amino acid variants in metagenomic read recruitment results),
* [anvi-gen-variability-network](https://anvio.org/m/anvi-gen-variability-network) (which is a program that turns an anvi'o nucleotide variability report into a Gephi compatible XML network file).

But by accessing anvi'o libraries directly, we get to apply filtering rules that are dependent on sample names, such as removing infants from the dataset whose mother does not have a metagenome and vice versa. One could indeed implement those steps after getting the necessary output files from `anvi-gen-variability-profile` using R, EXCEL or by manually selecting samples to be considered for the analysis. But a Pythonic approach helps with reproducibility, and reduces human error.

The actual bottom line is the following: these analyses can be done with anvi'o without writing a single line of Python code, too. In addition to reproducibilty, a side purpose of this workflow is to show those who might be interested in exploring anvi'o deeper what else can be done with it. So here we are.

Our analysis starts with importing some libraries that will be necessary later (of course, at the beginnign of the analysis we didn't know which libraries were necessary, but we expanded this section as we made progress with the code):

In [ ]:
import argparse
import pandas as pd

import anvio.terminal as terminal
import anvio.filesnpaths as filesnpaths

from anvio.variabilityops import NucleotidesEngine
from anvio.utils import store_dataframe_as_TAB_delimited_file

run = terminal.Run(width=27)

Here we define a Python function that takes a  set of anvi'o proifle-db and contigs-db files, and returns a comprehensive dictionary for the nucleotide variation per position:

In [ ]:
def get_snvs(contigs_db_path, profile_db_path):
    args = argparse.Namespace(contigs_db=contigs_db_path,
                              profile_db=profile_db_path,
                              gene_caller_ids='0',
                              compute_gene_coverage_stats=True)

    n = NucleotidesEngine(args, r=terminal.Run(verbose=False), p=terminal.Progress(verbose=False))
    n.process()

    return n.data

Please note that this is done by passing a set of arguments to the `NucleotidesEngine` class that we imported from `anvio.variabilityops`. We learned what to import from ths library and how to format the `args` from the source code of `anvi-gen-variability-profile` program. Please also note the `gene_caller_ids=0` directive among the arguments passed to the class. The gene caller id `0` corresponds to the mobA gene in pBI143. Since our intention was to focus on mobA, here we limit all data we will acquire from the `NucleotidesEngine` class to that gene.

The next function is a more complex one, but its sole purpose is to work with the sample names (which include information such as the unique family identifier, or whether a given sample belongs to the mother or an infant, etc) to summarize information about the mother-infant pairs remaining in our dataset, which is passed to the function as `udf`:

In [ ]:
def mother_infant_pair_summary(udf, drop_incomplete_families=False):
    # learn all the sample names that are present in the dataframe
    sample_names = list(udf.sample_id.unique())

    def get_countries(family_names):
        # learn which countries are present in the datsaet by
        # splitting that piece of information from sample names
        country_names = [s.split('_')[0] for s in family_names]
        return dict(zip(list(country_names), [country_names.count(i) for i in country_names]))

    # we will create this intermediate dictionary to resolve sample names into
    # some meaningful information about mother-infant pairs
    mother_infant_pairs = {}
    for sample_name in sample_names:
        family_name = '_'.join(sample_name.split('_')[0:2])[:-1]
        participant = sample_name.split('_')[1][-1]
        day = sample_name.split('_')[2]

        if family_name not in mother_infant_pairs:
            mother_infant_pairs[family_name] = {'M': [], 'C': []}

        mother_infant_pairs[family_name][participant].append(day)

    # now we can learn about family names that include at least one mother
    # and one infant:
    family_names_with_both_M_and_C = [p for p in mother_infant_pairs if mother_infant_pairs[p]['M'] and mother_infant_pairs[p]['C']]

    # now we subset the broader dictionary to have one that only includes
    # complete families (this is largely for reporting purposes)
    mother_infant_pairs_with_both_mother_and_infant = {}
    for family_name in family_names_with_both_M_and_C:
        mother_infant_pairs_with_both_mother_and_infant[family_name] = mother_infant_pairs[family_name]

    # report some reports .. these will be printed to the user's screen
    # when this function is called
    run.info('Num entires', f"{len(udf.index)}")
    run.info('Num samples', f"{udf.sample_id.nunique()}")
    run.info('Num families', f"{len(mother_infant_pairs)} / {get_countries(mother_infant_pairs.keys())}")
    run.info('   w/both members', f"{len(mother_infant_pairs_with_both_mother_and_infant)} / {get_countries(mother_infant_pairs_with_both_mother_and_infant.keys())}")

    if drop_incomplete_families:
        # reconstruct the original sample names for complete families:
        sample_names_for_families_with_both_M_and_C = []
        for family_name in mother_infant_pairs_with_both_mother_and_infant:
            for day in mother_infant_pairs_with_both_mother_and_infant[family_name]['M']:
                sample_names_for_families_with_both_M_and_C.append(f"{family_name}M_{day}")
            for day in mother_infant_pairs_with_both_mother_and_infant[family_name]['C']:
                sample_names_for_families_with_both_M_and_C.append(f"{family_name}C_{day}")

        # return a new dataframe after dropping all sample names from incomplete families
        # so we have a dataframe that is clean and reprsent only complete families (sad)
        return udf[udf['sample_id'].isin(sample_names_for_families_with_both_M_and_C)]
    else:
        pass

Now it is time to get single-nucleotide variant data for the mother-infant pairs from Finland, Sweden, USA, and Italy using the anvi'o project files we downloaded using the data pack before, using the fancy function `get_snvs` we defined above.

It is important to note that the resulting data frames will contain information only for samples that include at least one SNV in the mobA gene of the plasmid. This means, samples that have no SNVs will not be reported in the following dataframes even though they were in the original dataset.

In [ ]:
df_fin = get_snvs('fin/CONTIGS.db', 'fin/PROFILE.db')

df_swe = get_snvs('swe/CONTIGS.db', 'swe/PROFILE.db')

df_usa = get_snvs('usa/CONTIGS.db', 'usa/PROFILE.db')

df_ita = get_snvs('ita/CONTIGS.db', 'ita/PROFILE.db')

# combine all data frames
df = pd.concat([df_fin, df_swe, df_usa, df_ita])

# reset the `entry_id` column to make sure each entry has a
# unique identifier:
df['entry_id'] = range(0, len(df))

Now we take a quick look at the resulting dataframe that combines all data from all samples by simply sending this dataframe to the function `mother_infant_pair_summary` implemented above, so you can appreciate its true utility:

In [ ]:
mother_infant_pair_summary(df)

The dataframe `df` is quite a comprehensive one, as anvi'o generates an extremely rich output file to describe variants observed in metagenomes. We can take a very quick look, and go through the columns to have an idea (they scroll right):

In [ ]:
df

While this is our raw dataframe that contains all variants from all samples that had at least one variable nucleotide position in the mobA gene, it needs some cleaning.

For instance, in some samples the mobA gene will have a pretty low coverage to perform a robus analysis of SNV patterns, and such samples should be removed first. Here are some of the samples that have the lowest coverage values:

In [ ]:
sorted(df.gene_coverage.unique())[:20]

For a very stringent analysis, here we can drop samples from our dataframe where the mobA gene has less than 50X coverage:

In [ ]:
dfx = df.drop(df[df.gene_coverage < 50].index)

The new dataframe `dfx` only contains samples that have enough reads to cover mobA at 50X or more. But as you can imagine, this removal step does not include any logic to 'maintain' families in the dataset. Following the removal of samples based on the coverage of mobA, now there will be some infants without mother samples, and some mother samples with no infants. Since the purpose of this analysis to investigate plasmid transfer through SNVs, we don't need those samples.

And `mother_infant_pair_summary` comes to our rescue once again, as we request this function to further drop samples from `dfx` that belong to 'incomplete' families:

In [ ]:
dfx = mother_infant_pair_summary(dfx, drop_incomplete_families=True)

In our final dataset, we have 49 families for which at least one mother metagenome and one infant metagenome was present.

At this point we can report this final dataframe as a TAB-delmited file, an output that will be identical to an output `anvi-gen-variability-profile` would have generated:

In [ ]:
# this is the file name in which we will store all the
# final varaibility data:
variability_profile_path = "pBI143_SNVs.txt"

dfx.reset_index(drop=True, inplace=True)
dfx["entry_id"] = dfx.index

# order by [corresponding_gene_call, codon_order_in_gene]
dfx = dfx.sort_values(by = ["corresponding_gene_call", "codon_order_in_gene"])

# ask anvi'o to store it:
store_dataframe_as_TAB_delimited_file(dfx,
                                      variability_profile_path,
                                      columns=dfx.columns.tolist())

If you wish, you can take a look at it in your terminal.

The next step is to represent the information in this file as a network so we can visualize the relationships between all samples in the dataset with respect to their shared SNVs using Gephi. But first, we would like to generate a dictionary with sample information using sample names, so we can highlight samples that belong to the same family, etc.

For this, we will parse the sample names, and create a dictionary, `sample_information_dict`, to pass to the `VariabiltyNetwork` class below:

In [ ]:
# an empty dictionary
sample_information_dict = {}

# get the final sample names from `dfx`
sample_names = list(dfx.sample_id.unique())

# go through each sample name, split it into pieces,
# based on the `_` character, and fill in the
# dictionary
for sample_name in sample_names:
    family_name = '_'.join(sample_name.split('_')[0:2])[:-1]
    participant = sample_name.split('_')[1][-1]
    day = sample_name.split('_')[2]
    country = family_name.split('_')[0]
    
    sample_information_dict[sample_name] = {'family_name': family_name,
                                            'participant': participant,
                                            'country': country,
                                            'sample_name': sample_name,
                                            'coverage': dfx[dfx.sample_id == sample_name].gene_coverage.tolist()[0]}

The resulting is a very simple data structure that looks like this:

In [ ]:
sample_information_dict

Now it is time to ask anvi'o to convert all this into an XML file:

In [ ]:
from anvio.variabilityops import VariabilityNetwork

variability_network_path = "pBI143_SNVs.gexf"

args = argparse.Namespace(input_file=variability_profile_path,
                          include_competing_NTs='noise-robust',
                          output_file=variability_network_path)

variability_network = VariabilityNetwork(args, r=terminal.Run(verbose=False), p=terminal.Progress(verbose=False))
variability_network.samples_information_dict = sample_information_dict
variability_network.generate()

You can now open the resulting XML file, [`pBI143_SNVs.gexf`](https://merenlab.org/data/pBI143/files/pBI143_SNVs.gexf), in [Gephi](https://gephi.org/) to visualize this network.